In [100]:
def RandomAgent(env):
    '''
    Choosing a random move every turn.
    '''

    import random
    mtype = random.randint(1,2)
    direction = random.choice(env.direction_list)
    id = None
    if mtype == 1:
        unique_id = []
        for piece in env.pieces:
            if piece == None:
                continue
            if piece['position'] == (-1,-1):
                unique_id.append(piece['id'])
        if len(unique_id) == 0:
            return RandomAgent(env)
        id = random.choice(unique_id)
    else:
        unique_id = []
        for rows in env.board:
            for id in rows:
                if id == 0:
                    continue
                if id not in unique_id:
                    if env.pieces[int(id)]['color'] == env.player_turn:
                        unique_id.append(id)
        if len(unique_id) == 0:
            return RandomAgent(env)
        id = random.choice(unique_id)
    return mtype, id, direction

In [101]:
def CheckTrapAgent(env):
    '''
    Checking if there is taking a spawner possibility, 
    otherwise choosing a random move every turn.
    '''
    
    # Check if player can capture the Spawner piece
    mtype = 2
    for rows in env.board:
        for id in rows:
            if id == 0:
                continue
            if env.pieces[int(id)]['color'] == env.player_turn:
                for direction in env.direction_list:
                    capture_list = env.check_capture(mtype, id, direction)
                    if (3 - env.player_turn) in capture_list:
                        return mtype, id, direction

    import random
    mtype = random.randint(1,2)
    direction = random.choice(env.direction_list)
    id = None
    if mtype == 1:
        unique_id = []
        for piece in env.pieces:
            if piece == None:
                continue
            if piece['position'] == (-1,-1):
                unique_id.append(piece['id'])
        if len(unique_id) == 0:
            return RandomAgent(env)
        id = random.choice(unique_id)
    else:
        unique_id = []
        for rows in env.board:
            for id in rows:
                if id == 0:
                    continue
                if id not in unique_id:
                    if env.pieces[int(id)]['color'] == env.player_turn:
                        unique_id.append(id)
        if len(unique_id) == 0:
            return RandomAgent(env)
        id = random.choice(unique_id)
    return mtype, id, direction

In [112]:
def NegaMaxAgent(env):
    '''
    Simple Agent checking with depth 3. 
    The agent will try to save the Spawner.
    A good benchmark will be to defeat this agent.
    (This version is quite slow to run)
    '''
    
    def find_valid_moves(curr_env):
        possible_moves = []
        
        # mtype = 1
        for piece in curr_env.pieces:
            if piece == None:
                continue
            if piece['position'] == (-1,-1):
                for direction in curr_env.direction_list:
                    id = piece['id']
                    if curr_env.check_move(1, id, direction):
                        possible_moves.append((1, id, direction))
        
        # mtype = 2
        for piece in curr_env.pieces:
            if piece == None:
                continue
            id = piece['id']
            if piece['color'] == curr_env.player_turn:
                for direction in curr_env.direction_list:
                    if curr_env.check_move(2, id, direction):
                        possible_moves.append((2, id, direction))
        
        return possible_moves
    
    import copy
    import random
    def negamax_tree(curr_env, depth = 2, player_sign = 1):
        if depth == 0:
            return 0, (-1,-1,-1)
            
        if curr_env.winner != 0:
            return -player_sign, (-1,-1,-1)

        max_reward = 0
        best_move = []

        possible_moves = find_valid_moves(curr_env)
        for mtype, id, direction in possible_moves:
            nxt_env = copy.deepcopy(curr_env)
            valid = nxt_env.move(mtype, id, direction, check_validity = False)
            if not valid:
                continue
            reward, move = negamax_tree(nxt_env, depth-1, -player_sign)
            if reward > max_reward:
                max_reward = reward
                best_move = [(mtype, id, direction)]
            elif reward == max_reward:
                best_move.append((mtype, id, direction))

        return max_reward, random.choice(best_move)
    
    max_reward, best_move = negamax_tree(env)
    return best_move

In [118]:
def HeuristicAgent_v1(env):

    def find_valid_moves(curr_env):
        possible_moves = []
        
        # mtype = 1
        for piece in curr_env.pieces:
            if piece == None:
                continue
            if piece['position'] == (-1,-1):
                for direction in curr_env.direction_list:
                    id = piece['id']
                    if curr_env.check_move(1, id, direction):
                        possible_moves.append((1, id, direction))
        
        # mtype = 2
        for piece in curr_env.pieces:
            if piece == None:
                continue
            id = piece['id']
            if piece['color'] == curr_env.player_turn:
                for direction in curr_env.direction_list:
                    if curr_env.check_move(2, id, direction):
                        possible_moves.append((2, id, direction))
        
        return possible_moves
    
    # Edit Heuristic Valuation here
    def calc_value_board(curr_env, player_turn):
        value = 0
        if curr_env.winner != 0:
            value += 100
        
        for piece in curr_env.pieces:
            if piece == None:
                continue
            if piece['color'] == player_turn:
                value += 10
                for direction in curr_env.direction_list:
                    capture_list = curr_env.check_capture(2, piece['id'], direction)
                    if (3 - player_turn) in capture_list:
                        value += 50
            else:
                value -= 15
                for direction in curr_env.direction_list:
                    capture_list = curr_env.check_capture(2, piece['id'], direction)
                    if player_turn in capture_list:
                        value -= 50
        return value
        
    
    import copy
    
    possible_moves = find_valid_moves(env)
    best_move = (-1,-1,-1)
    best_value = -100
    for mtype, id, direction in possible_moves:
        curr_env = copy.deepcopy(env)
        curr_env.move(mtype, id, direction)
        value = calc_value_board(curr_env, env.player_turn)
        if value > best_value:
            best_value = value
            best_move = (mtype, id, direction)

    return best_move

In [123]:
# Simulate a game
import InTrap_env
env = InTrap()
simulate_game(env, HeuristicAgent_v1, HeuristicAgent_v1, verbose = True, MAX_STEPS = 1000)

  X   X   X   X  SP2  X 
  X   X   X   X   X   X 
  X   X   X   X   X   X 
  X   X   X   X   X   X 
  X   X   X   X   X   X 
  X  SP1  X   X   X   X 

  X   X   X   X  SP2  X 
  X   X   X   X   X   X 
  X   X   X   X   X   X 
  X   X   X   X   X   X 
  X  SP1  X   X   X   X 
  X   X   X   X   X   X 

  X   X   X   X  SP2  X 
  X   X   X   X   X   X 
  X   X   X   X   X   X 
  X   X   X   X   X   X 
  X  SP1  X   X   X   X 
  X   X   X  FL2  X   X 

  X   X   X   X  SP2  X 
  X   X   X   X   X   X 
  X   X   X   X   X   X 
 FL1  X   X   X   X   X 
  X  SP1  X   X   X   X 
  X   X   X  FL2  X   X 

  X   X   X   X  SP2  X 
  X   X   X   X   X   X 
  X   X   X   X   X   X 
 FL1  X   X   X   X   X 
  X  SP1  X   X   X   X 
  X   X   X  FL2 JP2  X 

  X   X   X   X  SP2  X 
  X   X   X   X   X   X 
  X   X   X   X   X   X 
 FL1 JP1  X   X   X   X 
  X  SP1  X   X   X   X 
  X   X   X  FL2 JP2  X 

  X   X   X   X  SP2  X 
  X   X   X   X   X   X 
  X   X   X   X   X   X 
 FL1 JP1  X   X   X

KeyboardInterrupt: 